In [0]:
%run ../config

In [0]:
start_date = 20260222163919
end_date = 20260222163942


df = spark.sql(f"""
with old_data as (
  select 
    vehicle_id, 
    unit_id, 
    sensor_a * sensor_b * sensor_c * sensor_d as front_sensor_product,
    sensor_e * sensor_f * sensor_g * sensor_h as rear_sensor_product
    from kaninipro_catalog.dev.sensor_data_silver 
where loaded_dt = {start_date}
),

new_data as (
  select 
    vehicle_id, 
    unit_id, 
    sensor_a * sensor_b * sensor_c * sensor_d as front_sensor_product,
    sensor_e * sensor_f * sensor_g * sensor_h as rear_sensor_product
    from kaninipro_catalog.dev.sensor_data_silver 
where loaded_dt = {end_date}
),

detection_derived as (
select 
  A.vehicle_id, 
  A.unit_id,
  case 
    when A.front_sensor_product > B.front_sensor_product 
    and A.rear_sensor_product > B.rear_sensor_product 
    then 'both_increased'
    else null 
  end as detection
from old_data A
join new_data B 
on A.vehicle_id = B.vehicle_id
and A.unit_id = B.unit_id
)

select 
vehicle_id, 
unit_id,  {start_date} as start_date, 
{end_date} as end_date,
current_timestamp() as __insert_time from detection_derived 
where detection = 'both_increased'
""")

# df.write.mode("append").partitionBy("").format("delta").saveAsTable(gold_tbl_name)


In [0]:
df = spark.sql(f"""
with old_data as (
  select 
    vehicle_id, 
    unit_id, 
    sensor_a * sensor_b * sensor_c * sensor_d as front_sensor_product,
    sensor_e * sensor_f * sensor_g * sensor_h as rear_sensor_product
    from kaninipro_catalog.dev.sensor_data_silver 
where loaded_dt = {start_date}
),

new_data as (
  select 
    vehicle_id, 
    unit_id, 
    sensor_a * sensor_b * sensor_c * sensor_d as front_sensor_product,
    sensor_e * sensor_f * sensor_g * sensor_h as rear_sensor_product
    from kaninipro_catalog.dev.sensor_data_silver 
where loaded_dt = {end_date}
),

detection_derived as (
select 
  A.vehicle_id, 
  A.unit_id,
  case 
    when A.front_sensor_product > B.front_sensor_product 
    and A.rear_sensor_product > B.rear_sensor_product 
    then 'both_increased'
    else null 
  end as detection
from old_data A
join new_data B 
on A.vehicle_id = B.vehicle_id
and A.unit_id = B.unit_id
)

select 
  vehicle_id, 
  unit_id,  
  {start_date} as start_date, 
  {end_date} as end_date,
  date_format(current_timestamp(), 'yyyy-MM-dd HH:mm') as processed_time
from detection_derived 
where detection = 'both_increased'
""")

df.write.mode("append").partitionBy("processed_time").format("delta").saveAsTable(gold_tbl_name)